In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gradio
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 130.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.3 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import pandas as pd
import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from io import BytesIO
import base64
import matplotlib.pyplot as plt
import numpy as np
import collections
from sentence_transformers import SentenceTransformer
import nltk
import umap
import hdbscan
from sklearn.neighbors import NearestNeighbors

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import os
os.chdir('/content/drive/MyDrive/PLP Proj')

In [ ]:
sentiment_model = AutoModelForSequenceClassification.from_pretrained("results/checkpoint-4000")
sentiment_tokenizer = AutoTokenizer.from_pretrained("results/checkpoint-4000")

with open("steam_review_topic_model_improved.pkl", "rb") as f:
    topic_model = pickle.load(f)


reviews_df = pd.read_csv("final_data_v4.csv")

In [ ]:
from qa_model import QAModel
qa_model = QAModel()
qa_model.load_model("./RAG for RecSys")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Model loaded from ./RAG for RecSys
Current parameters: top_k=100, top_n=20, top_games=3, reviews_per_game=3


In [ ]:
# Topic auto-labeling function
def auto_label_reviews(reviews, progress_callback=None):

    if isinstance(reviews, pd.Series):
        reviews = reviews.tolist()
    elif isinstance(reviews, str):
        reviews = [reviews]

    if progress_callback:
        progress_callback(0.1)

    sentences_with_origin = []
    for i, review in enumerate(reviews):
        for sent in nltk.sent_tokenize(review):
            if len(sent.split()) > 4:
                sentences_with_origin.append((sent, i))

    if not sentences_with_origin:
        if progress_callback:
            progress_callback(1.0)
        return {i: None for i in range(len(reviews))}

    if progress_callback:
        progress_callback(0.2)

    processed_reviews, review_origins = zip(*sentences_with_origin)

    best_params = topic_model['best_params']
    best_clusters = topic_model['best_clusters']
    cluster_summary = topic_model['cluster_summary']
    embeddings = topic_model['embeddings']

    if progress_callback:
        progress_callback(0.3)

    model_st = SentenceTransformer('all-mpnet-base-v2')
    new_embeddings = model_st.encode(processed_reviews, show_progress_bar=False)

    if progress_callback:
        progress_callback(0.5)

    umap_model = umap.UMAP(
        n_neighbors=best_params['n_neighbors'],
        n_components=best_params['n_components'],
        metric='cosine',
        random_state=best_params['random_state']
    )

    original_umap_embeddings = umap_model.fit_transform(embeddings)

    if progress_callback:
        progress_callback(0.7)

    new_umap_embeddings = umap_model.transform(new_embeddings)

    nbrs = NearestNeighbors(n_neighbors=5).fit(original_umap_embeddings)
    distances, indices = nbrs.kneighbors(new_umap_embeddings)

    if progress_callback:
        progress_callback(0.8)

    predicted_labels = []
    for idx_set in indices:
        neighbor_labels = [best_clusters.labels_[i] for i in idx_set]
        label_counts = collections.Counter(neighbor_labels)
        if len(label_counts) > 1 and -1 in label_counts:
            del label_counts[-1]
        if label_counts:
            predicted_labels.append(label_counts.most_common(1)[0][0])
        else:
            predicted_labels.append(-1)

    label_map = dict(zip(cluster_summary['label_st1'], cluster_summary['label']))
    review_topics = {}

    if progress_callback:
        progress_callback(0.9)

    for i in range(len(reviews)):
        topics = []
        for pred_label, origin_idx in zip(predicted_labels, review_origins):
            if origin_idx == i and pred_label in label_map:
                topics.append(label_map[pred_label])

        if not topics:
            review_topics[i] = "General Topic"
        else:
            review_topics[i] = collections.Counter(topics).most_common(1)[0][0]

    if progress_callback:
        progress_callback(1.0)

    return review_topics


In [ ]:
def recommend_games(query):
    result = qa_model.generate_recommendations(query)

    html = "<h3>🎯 Recommended Games</h3>"

    for game, reasons in zip(result["top_games"], result["reasons"]):
        html += f"""
        <div style="border: 1px solid #ccc; border-radius: 8px; padding: 10px; margin: 10px 0;">
            <b style="font-size: 18px; color: #2c3e50;">🎮 {game}</b><br>
        """
        for r in reasons:
            html += f"<i>{r}</i><br>"
        html += "</div>"

    return html

def analyze_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
    scores = torch.softmax(outputs.logits, dim=1)
    sentiment = ["Negative", "Positive"][scores.argmax().item()]
    return sentiment, scores[0].tolist()

def plot_sentiment_pie(stats):
    labels = list(stats.keys())
    sizes = list(stats.values())
    colors = ['#ff6666', '#66cc66']

    fig, ax = plt.subplots()
    fig.set_size_inches(4, 4)
    ax.pie(
        sizes,
        labels=labels,
        autopct='%1.1f%%',
        colors=colors,
        startangle=140
    )
    ax.axis('equal')

    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)

    return f"<img src='data:image/png;base64,{img_str}' style='width: 300px; height: 300px;'/>"

# Topic Distribution Chart
def plot_topic_pie(stats):
    labels = list(stats.keys())
    sizes = list(stats.values())

    cmap = plt.cm.get_cmap('tab20', len(labels))
    colors = [cmap(i) for i in range(len(labels))]

    fig, ax = plt.subplots()
    fig.set_size_inches(7, 7)
    wedges, texts, autotexts = ax.pie(
        sizes,
        labels=None,
        autopct='%1.1f%%',
        colors=colors,
        startangle=90
    )

    if len(labels) > 10:
        short_labels = [f"{l[:30]}..." if len(l) > 30 else l for l in labels]
        legend_elements = [plt.Line2D([0], [0], color=colors[i], lw=4, label=f"{short_labels[i]} ({sizes[i]})")
                         for i in range(min(10, len(labels)))]
        if len(labels) > 10:
            legend_elements.append(plt.Line2D([0], [0], color='gray', lw=4,
                                              label=f"Other Topics ({sum(sizes[10:])})"))
        ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5), fontsize=8)
    else:
        legend_elements = [plt.Line2D([0], [0], color=colors[i], lw=4, label=f"{labels[i]} ({sizes[i]})")
                         for i in range(len(labels))]
        ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))

    ax.axis('equal')
    plt.tight_layout()

    buf = BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)

    return f"<img src='data:image/png;base64,{img_str}' style='width: 100%; max-width: 600px;'/>"

def analyze_reviews(game_name, progress=gr.Progress()):
    df = reviews_df[reviews_df['app_name'] == game_name]
    if df.empty:
        return f"<p>No reviews found for {game_name}.</p>"

    progress(0, desc="Starting analysis...")

    progress(0.1, desc="Performing sentiment analysis...")
    sentiment_stats = {"Positive": 0, "Negative": 0}
    reviews_with_sentiment = []

    total_reviews = len(df)
    for idx, text in enumerate(df['review_text']):
        sentiment, _ = analyze_sentiment(text)
        sentiment_stats[sentiment] += 1
        reviews_with_sentiment.append((idx, text, sentiment))
        if idx % 10 == 0:
            progress(0.1 + 0.3 * (idx / total_reviews), desc=f"Analyzed {idx}/{total_reviews} reviews (sentiment)...")

    progress(0.4, desc="Starting topic analysis...")
    topic_dict = auto_label_reviews(df['review_text'].tolist(), progress_callback=lambda p: progress(0.4 + 0.4 * p, desc="Topic modeling..."))

    progress(0.8, desc="Compiling results...")
    topic_stats = {}
    for idx, topic in topic_dict.items():
        if topic in topic_stats:
            topic_stats[topic] += 1
        else:
            topic_stats[topic] = 1

    topic_stats = dict(sorted(topic_stats.items(), key=lambda x: x[1], reverse=True))

    sample_df = df.sample(n=min(30, len(df)), random_state=42)

    color_map = {
        "Positive": "green",
        "Negative": "red",
    }

    progress(0.85, desc="Selecting diverse representative comments...")

    topic_groups = {}
    for idx, row in df.iterrows():
        topic = topic_dict.get(idx, "General Topic")
        if topic not in topic_groups:
            topic_groups[topic] = []

        sentiment, _ = analyze_sentiment(row['review_text'])
        topic_groups[topic].append((idx, row['review_text'], sentiment))

    detailed_reviews = []

    for topic in topic_stats.keys():
        if topic in topic_groups and len(topic_groups[topic]) > 0:
            idx, text, sentiment = topic_groups[topic][0]

            color = color_map.get(sentiment, "black")
            sentiment_tag = f"<span style='color:{color}; font-weight:bold'>{sentiment}</span>"
            topic_tag = f"<span style='color:blue; font-weight:bold'>{topic}</span>"

            detailed_reviews.append(f"{text}<br><b>Sentiment:</b> {sentiment_tag} | <b>Topic:</b> {topic_tag}")

            if len(detailed_reviews) >= 5:
                break

    if len(detailed_reviews) < 5:
        for topic in topic_stats.keys():
            if topic in topic_groups and len(topic_groups[topic]) > 1:
                for i in range(1, min(3, len(topic_groups[topic]))):
                    idx, text, sentiment = topic_groups[topic][i]

                    color = color_map.get(sentiment, "black")
                    sentiment_tag = f"<span style='color:{color}; font-weight:bold'>{sentiment}</span>"
                    topic_tag = f"<span style='color:blue; font-weight:bold'>{topic}</span>"

                    detailed_reviews.append(f"{text}<br><b>Sentiment:</b> {sentiment_tag} | <b>Topic:</b> {topic_tag}")

                    if len(detailed_reviews) >= 5:
                        break

                if len(detailed_reviews) >= 5:
                    break

    progress(0.9, desc="Generating visualizations...")
    sentiment_chart_html = plot_sentiment_pie(sentiment_stats)
    topic_chart_html = plot_topic_pie(topic_stats)

    html = f"<h2>🎮 Game: {game_name}</h2>"

    html += "<div style='display: flex; flex-wrap: wrap; justify-content: space-around;'>"
    html += "<div style='flex: 1; min-width: 300px;'>"
    html += "<h3>📊 Sentiment Distribution</h3>" + sentiment_chart_html
    html += "</div>"
    html += "<div style='flex: 1; min-width: 300px;'>"
    html += "<h3>🏷️ Topic Distribution</h3>" + topic_chart_html
    html += "</div>"
    html += "</div>"

    html += "<h3>📝 Representative Comments (Diverse Topics)</h3><ul>"
    for r in detailed_reviews:
        html += f"<li>{r}</li>"
    html += "</ul>"

    progress(1.0, desc="Analysis complete!")
    return html

def recommend_games_with_loading(query):
    yield gr.update(value="<p>⏳ Generating recommendations...</p>")
    result = recommend_games(query)
    yield gr.update(value=result)

def analyze_reviews_with_loading(game_name):
    return analyze_reviews(game_name)


In [ ]:
with gr.Blocks(title="🎮 Game Review Explorer") as demo:

    with gr.Tab("🎯 Game Search"):
        gr.Markdown("## 🔍 Game Recommendation Assistant")
        user_query = gr.Textbox(label="What kind of game are you looking for?", placeholder="For example: Are there any story-rich role-playing games?")
        query_submit = gr.Button("Recommend")
        query_output = gr.HTML()
        query_submit.click(
            fn=recommend_games_with_loading,
            inputs=user_query,
            outputs=query_output,
            queue=True
        )

    with gr.Tab("💬 Review Analysis"):
        gr.Markdown("## 🧠 Sentiment & Topic Analysis of Reviews")
        game_name_input = gr.Textbox(label="Enter game name, e.g., Counter-Strike")
        game_submit = gr.Button("Analyze")
        game_output = gr.HTML()
        game_submit.click(
            fn=analyze_reviews_with_loading,
            inputs=game_name_input,
            outputs=game_output,
            queue=True
        )

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7b228889e02d709a2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
